In [1]:
import pandas as pd
import tensorflow as tf
import codecs
import numpy as np
import math
import csv
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import recall_score, precision_score # New
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn import preprocessing
import keras
import matplotlib.pyplot as plt
from keras.callbacks import ModelCheckpoint, CSVLogger, EarlyStopping, LearningRateScheduler, ReduceLROnPlateau
from keras.layers import merge
from keras.models import Sequential, Model
from keras.layers import Dense, Input, Flatten, Add
from keras.layers import Activation, BatchNormalization, regularizers
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.merge import concatenate
from keras.layers import LSTM, Bidirectional
from keras.utils import plot_model
from keras.layers import Dropout
from scipy import signal
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
import openpyxl

from keras.backend import tensorflow_backend as K
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
K.set_session(tf.Session(config=config))

class LossHistory(keras.callbacks.Callback):  # history = LossHistory()
    def init(self):  # history.init()
        self.losses = []
        # self.accs = []
        self.val_losses = []
        # self.val_accs = []
        self.mses = []
        self.maes = []
        self.mapes = []
        self.val_mses = []
        self.val_maes = []
        self.val_mapes = []

    def on_epoch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))
        # self.accs.append(logs.get('acc'))
        self.val_losses.append(logs.get('val_loss'))
        # self.val_accs.append(logs.get('val_accuracy
        self.mses.append(logs.get('mean_squared_error'))
        self.maes.append(logs.get('mean_absolute_error'))
        self.mapes.append(logs.get('mean_absolute_percentage_error'))
        self.val_mses.append(logs.get('val_mean_squared_error'))
        self.val_maes.append(logs.get('val_mean_absolute_error'))
        self.val_mapes.append(logs.get('val_mean_absolute_percentage_error'))

def create_dataset(dataset, timesteps):
    dataX, dataY = [], []
    for i in range(len(dataset) - timesteps - 1):
        a = dataset[i:(i + timesteps), 0]
        dataX.append(a)
        dataY.append(dataset[i + timesteps, 0])
    return np.array(dataX), np.array(dataY)

def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

data_path = 'C:/Users/Smartgrid_ML/Desktop/KMS/climate_hour.csv'
data_source = pd.read_csv(data_path)
data_source = pd.DataFrame(data_source)
data_set = data_source.iloc[:, 2]
data = np.array(data_set)
data = data.reshape(-1, 1)
scaler = MinMaxScaler(feature_range=(0, 1))
data = scaler.fit_transform(data)
Maxvalue_1 = scaler.data_max_
print(data.shape)

Tr_mean = np.nanmean(data)
Tr_var = np.nanvar(data)

print('Dataset mean = ', Tr_mean, 'var = ', Tr_var)
print('Maximum value = ', Maxvalue_1)

Training_set = data[0:42022, :]
Validation_set = data[42023:63031, :]
Test_set = data[63032:, :]

features = 1
timesteps = 24
trainX, trainY = create_dataset(Training_set, timesteps)
validX, validY = create_dataset(Validation_set, timesteps)
testX, testY = create_dataset(Test_set, timesteps)

trainX = np.reshape(trainX, (trainX.shape[0], trainX.shape[1], 1))
validX = np.reshape(validX, (validX.shape[0], validX.shape[1], 1))
testX = np.reshape(testX, (testX.shape[0], testX.shape[1], 1))

# -----------------------------------------------------------------------------------------------------------
# -----------------------------------------------------------------------------------------------------------
import keras.backend.tensorflow_backend as K
with K.tf.device('/gpu:0'):
    
    visible1 = Input(shape=(timesteps, features))

    bi11 = (Bidirectional(LSTM(16, return_sequences=True)))(visible1)  ### success
    bi12 = (Bidirectional(LSTM(16, return_sequences=True)))(bi11)
    bi13 = (Bidirectional(LSTM(16, return_sequences=True)))(bi12)
    bi14 = (Bidirectional(LSTM(16, return_sequences=True)))(bi13)
    bi15 = (Bidirectional(LSTM(16, return_sequences=True)))(bi14)
    bi16 = (Bidirectional(LSTM(16, return_sequences=True)))(bi15)
    bi17 = (Bidirectional(LSTM(16, return_sequences=True)))(bi16)
    bi18 = (Bidirectional(LSTM(16, return_sequences=True)))(bi17)
    bi19 = (Bidirectional(LSTM(16, return_sequences=True)))(bi18)
    bi20 = (Bidirectional(LSTM(16, return_sequences=True)))(bi19)
    bi21 = (Bidirectional(LSTM(32)))(bi20)

    dense11 = Dense(24, activation='selu')(bi21)
    out1 = Dense(1, activation='selu')(dense11)

    model = Model(inputs=[visible1], outputs=[out1])  # multi-input, multi-output

#########################################

print(model.summary())
plot_model(model, to_file='C:/Users/Smartgrid_ML/Documents/KMS/DRNet-3_Structure.svg')
SVG(model_to_dot(model, show_shapes=True).create(prog='dot', format='svg'))

reducelr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=1)  # 에포크 1동안 좋아지지 않으면 호출

model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mse', 'mae', 'mape'])

batch_size = 100
epochs = 20

history = LossHistory()
history.init()

model.fit(trainX, trainY, epochs=epochs, batch_size=batch_size, shuffle=False, validation_data=(validX, validY), callbacks=[history, reducelr])  # , checkpoint

trainPredict = model.predict(trainX, batch_size=batch_size)
validPredict = model.predict(validX, batch_size=batch_size)
testPredict = model.predict(testX, batch_size=batch_size)

trainPredict = trainPredict.reshape([-1])
validPredict = validPredict.reshape([-1])
testPredict = testPredict.reshape([-1])
testY = testY.reshape([-1])
validY = validY.reshape([-1])
trainY = trainY.reshape([-1])

np.savetxt('trainPredict_5.txt', trainPredict)
np.savetxt('testPredict_5.txt', testPredict)
np.savetxt('validPredict_5.txt', validPredict)
np.savetxt('testY_5.txt', testY)
np.savetxt('validY_5.txt', validY)
np.savetxt('trainY_5.txt', trainY)

print('--------------------------------------------------------')
print(' history.mses(=loss) = ', history.mses)  # loss == mse
print(' history.maes        = ', history.maes)
print(' history.mapes        = ', history.mapes)
print(' history.val_mses(=loss)= ', history.val_mses)  # loss == mse
print(' history.val_maes       = ', history.val_maes)
print(' history.val_mapes       = ', history.val_mapes)
print('--------------------------------------------------------')

plt.figure(1)
plt.plot(testPredict, 'r')
plt.plot(testY, 'b')
plt.show()

plt.figure(2)
plt.plot(history.mses, 'b--')
plt.plot(history.val_mses, 'b:')
plt.plot(history.maes, 'ro')
plt.plot(history.val_maes, 'r+')
plt.ylabel('Loss and Accuracy')
plt.xlabel('epoch')
plt.show()

print('Test Score > MSE == ', mean_squared_error(testY, testPredict), ' MAE == ', mean_absolute_error(testY, testPredict), 'MAPE == ', mean_absolute_percentage_error(testY, testPredict))

Using TensorFlow backend.
W0110 14:37:43.781729 11872 deprecation_wrapper.py:119] From C:\Users\Smartgrid_ML\Anaconda3\envs\tensorflow-gpu\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0110 14:37:43.781729 11872 deprecation_wrapper.py:119] From C:\Users\Smartgrid_ML\Anaconda3\envs\tensorflow-gpu\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0110 14:37:43.795430 11872 deprecation_wrapper.py:119] From C:\Users\Smartgrid_ML\Anaconda3\envs\tensorflow-gpu\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



(70037, 1)
Dataset mean =  0.5363373411352942 var =  0.01966813341576947
Maximum value =  [37.28]


W0110 14:37:46.966321 11872 deprecation.py:323] From C:\Users\Smartgrid_ML\Anaconda3\envs\tensorflow-gpu\lib\site-packages\keras\backend\tensorflow_backend.py:3217: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 24, 1)             0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 24, 32)            2304      
_________________________________________________________________
bidirectional_2 (Bidirection (None, 24, 32)            6272      
_________________________________________________________________
bidirectional_3 (Bidirection (None, 24, 32)            6272      
_________________________________________________________________
bidirectional_4 (Bidirection (None, 24, 32)            6272      
_________________________________________________________________
bidirectional_5 (Bidirection (None, 24, 32)            6272      
_________________________________________________________________
bidirectional_6 (Bidirection (None, 24, 32)            6272      
__________

W0110 14:37:51.500619 11872 deprecation_wrapper.py:119] From C:\Users\Smartgrid_ML\Anaconda3\envs\tensorflow-gpu\lib\site-packages\keras\optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0110 14:38:01.157714 11872 deprecation_wrapper.py:119] From C:\Users\Smartgrid_ML\Anaconda3\envs\tensorflow-gpu\lib\site-packages\keras\backend\tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.

W0110 14:38:01.844384 11872 deprecation_wrapper.py:119] From C:\Users\Smartgrid_ML\Anaconda3\envs\tensorflow-gpu\lib\site-packages\keras\backend\tensorflow_backend.py:973: The name tf.assign is deprecated. Please use tf.compat.v1.assign instead.



Train on 41997 samples, validate on 20983 samples
Epoch 1/20
41997/41997 [==============================] - 238s 6ms/step - loss: 0.0109 - mean_squared_error: 0.0109 - mean_absolute_error: 0.0818 - mean_absolute_percentage_error: 716.7462 - val_loss: 0.0164 - val_mean_squared_error: 0.0164 - val_mean_absolute_error: 0.1040 - val_mean_absolute_percentage_error: 19.9368
Epoch 2/20
41997/41997 [==============================] - 228s 5ms/step - loss: 0.0118 - mean_squared_error: 0.0118 - mean_absolute_error: 0.0855 - mean_absolute_percentage_error: 11712.2477 - val_loss: 0.0169 - val_mean_squared_error: 0.0169 - val_mean_absolute_error: 0.1079 - val_mean_absolute_percentage_error: 22.6663
Epoch 3/20
41997/41997 [==============================] - 229s 5ms/step - loss: 0.0248 - mean_squared_error: 0.0248 - mean_absolute_error: 0.1301 - mean_absolute_percentage_error: 13533.7465 - val_loss: 0.0162 - val_mean_squared_error: 0.0162 - val_mean_absolute_error: 0.1053 - val_mean_absolute_percentag

<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

Test Score > MSE ==  0.0008310659956060693  MAE ==  0.02195041727659361 MAPE ==  3.946298580183479
